In [1]:
import os
os.chdir(r"D:/myGit/github/facenet/src")
print(os.getcwd())

D:\myGit\github\facenet\src


In [2]:
import sys 
sys.path.append("../contributed")
import face
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2
%matplotlib inline

# 计算特征向量 512d f32ndarray

In [3]:
face_image1=cv2.imread("X:/share/image/mydataset/test_img/amellanby/amellanby1.jpg")
%prun faceRecognition=face.Recognition()




Model directory: X:/share/model/20180402-114759
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from X:/share/model/20180402-114759\model-20180402-114759.ckpt-275
 

In [5]:
%time face1=faceRecognition.add_identity(face_image1,"amellanby")
#face1[0].embedding

print(face1[0].bounding_box)


Wall time: 252 ms
[ 70  94 329 411]


* 比对速度平均为6.67us \次 这是否意味着，classifier 是快速查询，实际上一般情况下可以通过遍历完成1：N识别？

In [11]:
# 距离
import numpy as np
#
face_image2=cv2.imread("X:/share/image/mydataset/test_img/andrew/andrew10.jpg")
face2=faceRecognition.add_identity(face_image2,"andrew10")

emb1=face1[0].embedding
emb2=face2[0].embedding
%timeit dist = np.sqrt(np.sum(np.square(np.subtract(emb1, emb2))))
print(dist)  

6.67 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.0174652


# API

In [ ]:
# 初始化
# 载入配置文件
import os
os.chdir(r"D:/myGit/github/facenet/src")
print(os.getcwd())
import sys 
sys.path.append("../contributed")
import face
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2
faceRecognition=face.Recognition()

## 特征提取接口
* 一次数据量为6.79kbytes

In [96]:
import sys
sys.path.append(r"C:\Users\user\OneDrive\code space\python\1notebook")
from decoration import OutputRedirect
from kafka import KafkaProducer
producerInfoMap={"producer": KafkaProducer(bootstrap_servers="192.168.1.201:9092"),"topic":"test"}

@OutputRedirect.output2kafka(**producerInfoMap)
@kvFaceEmbedding
def generateEmbedding():
    face_image1=cv2.imread("X:/share/image/mydataset/test_img/andrew/andrew10.jpg")
    face1=faceRecognition.add_identity(face_image1,"andrew")
    assert len(face1)==1,"more then one face detect"    
    return {"id":face1[0].name,"embedding":face1[0].embedding}
generateEmbedding()

<kafka.producer.kafka.KafkaProducer object at 0x0000012AA8D328D0> test


ValueError: too many values to unpack (expected 2)

In [94]:
# 数据转换
key=bytes(face1[0].name,"utf-8")
assert type(key) is bytes
emb=bytes(face1[0].embedding)
assert type(emb) is bytes
#print(list(face1[0].embedding))
bytes( str([1.2,3]) ,"utf-8")
#print(emb)
import numpy as np
import json
def kvFaceEmbedding(functionName):
    """
    dict -> bytes
    """
    def func_in(*args, **kwargs):

        ret = functionName(*args, **kwargs)
        assert type(ret["id"]) is str,"id is not string "
        assert type(ret["embedding"]) is np.ndarray
        ret["embedding"].dtype=float
        ret["embedding"]=list(ret["embedding"])
        return bytes(json.dumps(ret),"ascii")
    return func_in
@kvFaceEmbedding
def t():
    return {"id":face1[0].name,"embedding":face1[0].embedding}
type(t())
#face1[0].embedding

bytes

In [87]:
face1[0].embedding.dtype="float"
len(face1[0].embedding)

256

# 优化单次运算时间
不要重复实例化 Encode！Encode 中的所有方法在实例化时都耗费大量时间。
所以首先要把实例对象载入内存

In [33]:
import tensorflow as tf
import facenet
from face import facenet_model_checkpoint
sess = tf.Session()
with sess.as_default():
    %prun facenet.load_model(facenet_model_checkpoint)
"""需要明白这2个函数 并且少调用
 saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file), input_map=input_map)
        saver.restore(tf.get_default_session(), os.path.join(model_exp, ckpt_file))
"""


Model directory: X:/share/model/20180402-114759
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from X:/share/model/20180402-114759\model-20180402-114759.ckpt-275
 